In [1]:
import pandas as pd
from unidecode import unidecode
import numpy as np
import re
import importlib
import global_settings
importlib.reload(global_settings)

<module 'global_settings' from '/data/workdirectory/03. HMS unification/5. 20250320/MC/global_settings.py'>

In [2]:
setts = global_settings.Settings()

Directory found: /data/workdirectory/03. HMS unification/5. 20250320/MC/input
Directory found: /data/workdirectory/03. HMS unification/5. 20250320/MC/input/tmp1
Directory found: /data/workdirectory/03. HMS unification/5. 20250320/MC/input/tmp2
Directory found: /data/workdirectory/03. HMS unification/5. 20250320/MC/output


In [3]:
df = pd.read_csv(setts.tmp1_folder + '/cleaned_hms.csv',dtype=str).fillna('') # file moi can unification
province = pd.read_csv(setts.input_folder + '/province.csv',dtype=str).fillna('')
district = pd.read_csv(setts.input_folder + '/district.csv',dtype=str).fillna('')


In [4]:
df

,customer_code,customer_name,mobile_phone,customer_address,vin,head_code,city,is_company
0,1-23W4GGY,THỌ CHÚ,0918118098,NGÃI I TAM NGÃI CẦU KÈ TRÀ VINH TRÀ VINH,RLHJC5350DY043924,42034,TRÀ VINH,0
1,1-6I663QL,LÊ THÀNH TÀI,0907104038,141 6 ẤP 7 XÃ TÂN HIỆP HÓC MÔN HỒ CHÍ MINH HỒ ...,RLHKF5308SZ300075,42034,HỒ CHÍ MINH,0
2,1-3SFXBRK,TOÀN ANH,0908678461,TÂN THÔNG HỘI ẤP TÂN TIẾN CỦ CHI HỒ CHÍ MINH H...,RLHJF4601DY597619,42034,HỒ CHÍ MINH,0
3,1-1ZC6N3B,VĂN SỰ HỒ,0932622942,TỔ 9 ẤP 2A TÂN THẠNH ĐÔNG CỦ CHI HỒ CHÍ MINH H...,RLHKF4211MY015800,42034,HỒ CHÍ MINH,0
4,1-55F-14132,TRẦN ĐÌNH QUÂN,0933544794,47 22 KP4A TÂN THỚI HIỆP HỒ CHÍ MINH,RLHJF4611EY828738,42034,HỒ CHÍ MINH,0
...,...,...,...,...,...,...,...,...
211309,1-6A9CA3E,LÊ VĂN THẮNG,0988422595,ĐẠI LỘC TT BÚT SƠN HOẰNG HÓA THANH HÓA THANH HÓA,RLHJK0380RY047043,21014,THANH HÓA,0
211310,1-GU5-3518,CHU THỊ HIỆP,0973055785,THÁNG 10 HOẰNG PHỤ THANH HÓA,RLHJK0347RZ412292,21014,THANH HÓA,0
211311,1-GVM-4759,HOÀNG THỊ QUYÊN,0388061120,Đ6 HOẰNG VINH THANH HÓA,RLHJF5804EY506671,21014,THANH HÓA,0
211312,1-FVS-8405,NGUYỄN THỊ NGA,0868389081,ÐÔNG XUÂN TRƯỜNG TRUNG THANH HÓA,RLHJF5800EY243577,21014,THANH HÓA,0


In [5]:
df.insert(0, 'unq_id', range(0, len(df)))

In [6]:
hms = df[['unq_id','customer_code', 'customer_name', 'mobile_phone', 'customer_address','vin']]
hms = hms.rename(columns= {'customer_code' : 'id',
                        'customer_name' : 'name',
                        'mobile_phone' : 'phone',
                        'customer_address' : 'address',
                        'vin' : 'vin'})
# hms.insert(0, 'unq_id', range(0, len(hms))) # ??????????
hms['address_cleansing'] = hms['address'].apply(lambda x: x if x == '' else unidecode(x).lower())

In [7]:
def remove_special_characters(text):
    new_text = ''
    for i in unidecode(text):
        if re.search(r'^[A-Za-z0-9\s]+$',i) is None:
            new_text = new_text + ' '
        else:
            new_text = new_text + i
    return new_text


def clean_city(text):
    word_lst = (' dl ',' tt ',' kp ',' tx ',' tp ',' tl ',' ql ',' h ',' t ',' d ',' x ',' p ',' q ',' k ',' vn','dai lo ','thi tran ','thi xa ','khu pho ','thanh pho ','tinh lo ','quoc lo ','huyen ','tinh ','duong ','xa ','phuong ','quan ','khom ','viet nam','thon ','so nha ','sn ')
    result = text.strip()
    if len(result) > 2:
        if text[:2].startswith(('h ', 't ', 'd ', 'x ', 'p ', 'q ', 'k ', 'sn', 'tp')):
            result = text[2:]
    
    if len(result) > 3:
        if text[:3].startswith(('dl ', 'tt ', 'kp ', 'tx ', 'tp ', 'tl ', 'ql ', 'so ')):
            result = text[3:]
    for word in word_lst:
        result = result.replace(word,' ')
    return result

def extract_city_from_address(address,city_list):
    len_city = 0
    city = ''
    for city_name in city_list:
        if re.search(r'%s$' %(city_name),address):
            if len(city_name) >= len_city:
                len_city = len(city_name)
                city = city_name
    if city == '':
        return 'other'
    return city

def extract_district_from_address(city,address,temp_district):
    if city == 'other':
        return 'other'
    else:
        district_fillter = temp_district[temp_district['city_name'] == city]
        len_district = 0
        district = 'other'
        for i in district_fillter['code_name']:
            if re.search(r'%s$' %(i),address):
                if len(i) >= len_district:
                    len_district = len(i)
                    district = i
        return district
    
def cleansing_phone(x): # ca
    x = str(x)
    if x[:2] == '84':
        x = '0' + x[2:]
    elif x[:1] != '0':
        x = '0' + x
    return x

def convert_phone(phone,temp_convert_phone):
    phone = str(phone)
    temp_convert_phone = temp_convert_phone.astype(str)
    new_phone = ''
    for i in range(len(temp_convert_phone)):
        len_old_number = len(temp_convert_phone['old_number'][i])
        if phone[:len_old_number] == temp_convert_phone['old_number'][i]:
            new_phone = temp_convert_phone['new_number'][i] + phone[len_old_number:]
    if new_phone == '':
        return phone
    else:
        return new_phone
    
def update_city_cleansing(x,province):
    city = ''
    len_city = 0
    for i in province[province['name_acr'] == x]['code_name']:
        if len (i) >  len_city:
            city = i
            len_city = len(i)
    if city:
        return city
    else:
        return x

In [8]:
hms['address_cleansing'] = hms['address_cleansing'].apply(lambda x: x if x is np.nan else clean_city(remove_special_characters(x)).replace(' ',''))
# len_max = province.code_name.str.len().max()

#tach thanh pho khoi dia chi
hms['city'] = hms['address_cleansing'].apply(lambda x: 'other' if x is np.nan else extract_city_from_address(x,province['code_name']))
hms['address_cleansing'] = hms.apply(lambda x: re.sub(r'%s$'%(x['city']),'', x['address_cleansing']) if x['city'] != 'other' else x['address_cleansing'] ,axis=1)

#map cac quan/huyen voi tinh/thanh pho
temp_district = district[['code_name','name_acr','province_code']].drop_duplicates()
temp_district = temp_district[temp_district['name_acr'] != temp_district['code_name']]
temp_district = temp_district.merge(province[['code_name','code']],left_on='province_code',right_on='code')
temp_district.rename(columns={'code_name_x' : 'code_name', 'code_name_y' : 'city_name'}, inplace=True)

#tach quan/huyen khoi dia chi
hms['district'] = hms.apply(lambda x: extract_district_from_address(x['city'],x['address_cleansing'],temp_district),axis=1)
hms['address_cleansing'] = hms.apply(lambda x: re.sub(r'%s$'%(x['district']),'', x['address_cleansing']) if (x['city'] != 'other' and x['district'] != 'other') else x['address_cleansing'] ,axis=1)

In [9]:
#tao bang combination theo code goc nhung khong biet de lam gi
combination = hms[['unq_id','id','vin','name','address_cleansing','district','city','phone']]
combination = combination.drop_duplicates()
combination.sort_values(by=['id'],inplace=True)
combination.insert(0, 'System_id', range(0, len(combination))) #?????

combination = combination.rename(columns= {'name' : 'CUSTOMER_NAME',
                                            'phone' : 'MOBILE_PHONE',
                                            'address_cleansing' : 'CUSTOMER_ADDRESS',
                                            'district' : 'DISTRICT',
                                            'city' : 'CITY',
                                            'vin' : 'VIN'})

In [10]:
#tao cac cot cleansing va loai bo ky tu dac biet + dau tieng viet
lst_col = ['unq_id','id',"System_id", "VIN", "CUSTOMER_NAME", "CUSTOMER_ADDRESS", "DISTRICT", "CITY", "MOBILE_PHONE"]
temp_cleansing = combination[lst_col]
for col in lst_col:
    # col_name = re.search(r'phone|vin|name|address|city|district',col, re.IGNORECASE)
    col_name = re.search(r'address|city|district',col, re.IGNORECASE)
    if  col_name:
        col_name = col_name[0].lower() + "_cleansing"
        if re.search(r'phone|vin',col_name,re.IGNORECASE):
            temp_cleansing[col_name] = temp_cleansing[col].apply(lambda x: remove_special_characters(x).replace(' ','') if x is not np.nan else x)
        else:
            temp_cleansing[col_name] = temp_cleansing[col].apply(lambda x: unidecode(x).lower() if x is not np.nan else x)


#clean city
temp_cleansing['city_cleansing'] = temp_cleansing['city_cleansing'].apply(lambda x: x[:x.find('(')] if (x.find('(') != -1) and (x is not np.nan) else x)
temp_cleansing['city_cleansing'] = temp_cleansing['city_cleansing'].apply(lambda x: x.replace('app','') if x is not np.nan else x)


#clean name va lai clean address,city,district 1 lan nua
# lst_col_2 = ['name_cleansing','address_cleansing','district_cleansing','city_cleansing']
lst_col_2 = ['address_cleansing','district_cleansing','city_cleansing']
for col in lst_col_2:
    if col == 'name_cleansing':
        temp_cleansing[col] = temp_cleansing[col].apply(lambda x: remove_special_characters(x).replace(' ','') if x is not np.nan else x)
    else:
        temp_cleansing[col] = temp_cleansing[col].apply(lambda x: clean_city(remove_special_characters(x)).replace(' ','') if x is not np.nan else x)


temp_cleansing_col = temp_cleansing.columns.tolist()

#update cot city
temp_cleansing['city_cleansing'] = temp_cleansing['city_cleansing'].apply(lambda x: update_city_cleansing(x,province) if x is not np.nan else x)

temp_cleansing = temp_cleansing.merge(province[['code_name','code']],left_on='city_cleansing', right_on='code_name', how='left')
temp_cleansing = temp_cleansing.merge(district[['name_acr','province_code','code_name']], left_on=['district_cleansing','code'],right_on=['name_acr','province_code'], how='left')

#update cot district
temp_cleansing['district_cleansing'] = temp_cleansing.apply(lambda x: x['district_cleansing'] if x['code_name_y'] is np.nan else x['code_name_y'], axis=1)
temp_cleansing = temp_cleansing[temp_cleansing_col]

/tmp/ipykernel_9431/3691466335.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_cleansing[col_name] = temp_cleansing[col].apply(lambda x: unidecode(x).lower() if x is not np.nan else x)
/tmp/ipykernel_9431/3691466335.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_cleansing[col_name] = temp_cleansing[col].apply(lambda x: unidecode(x).lower() if x is not np.nan else x)
/tmp/ipykernel_9431/3691466335.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [11]:
#update cot address voi city
temp_cleansing = temp_cleansing.merge(province[['code_name','name_acr','code']], left_on = 'city_cleansing', right_on='code_name', how='left').fillna('')
temp_cleansing['address_cleansing'] = temp_cleansing.apply(lambda x: x['address_cleansing'].replace(x['name_acr'],'') if x['address_cleansing'] and x['name_acr'] else x['address_cleansing'],axis=1)
temp_cleansing['address_cleansing'] = temp_cleansing.apply(lambda x: x['address_cleansing'].replace(x['code_name'],'') if x['address_cleansing'] and x['code_name'] else x['address_cleansing'],axis=1)

In [12]:
#update cot address voi district
temp_cleansing = temp_cleansing.merge(temp_district[['code_name','name_acr','province_code']], left_on = ['district_cleansing','code'], right_on=['code_name','province_code'], how='left').fillna('')
temp_cleansing['address_cleansing'] = temp_cleansing.apply(lambda x: x['address_cleansing'].replace(x['name_acr_y'],'') if x['address_cleansing'] and x['name_acr_y'] else x['address_cleansing'],axis=1)
temp_cleansing['address_cleansing'] = temp_cleansing.apply(lambda x: x['address_cleansing'].replace(x['code_name_y'],'') if x['address_cleansing'] and x['code_name_y'] else x['address_cleansing'],axis=1)

In [13]:
# temp_cleansing = temp_cleansing[temp_cleansing_col]
# cleansing = temp_cleansing.drop_duplicates()

In [14]:
# cleansing['vin_group'] = cleansing['vin_cleansing'].apply(lambda x: x[3:8] if x else x)
# cleansing['phone_group'] = cleansing['phone_cleansing'].apply(lambda x: x[0:3] if x else x)
# cleansing = cleansing[["vin_group","vin_cleansing","name_cleansing","address_cleansing","district_cleansing","city_cleansing","phone_group","phone_cleansing"]]
# cleansing = cleansing.drop_duplicates()

In [15]:
clean_address_output = df.merge(temp_cleansing,on='unq_id',how='left')[['customer_code', 'customer_name', 'mobile_phone',
       'customer_address', 'vin', 'is_company','district_cleansing','city_cleansing']]
clean_address_output.rename(columns={'district_cleansing' : 'district',
                                   'city_cleansing' : 'city'}, inplace=True)

clean_address_output

,customer_code,customer_name,mobile_phone,customer_address,vin,is_company,district,city
0,1-23W4GGY,THỌ CHÚ,0918118098,NGÃI I TAM NGÃI CẦU KÈ TRÀ VINH TRÀ VINH,RLHJC5350DY043924,0,travinh,travinh
1,1-6I663QL,LÊ THÀNH TÀI,0907104038,141 6 ẤP 7 XÃ TÂN HIỆP HÓC MÔN HỒ CHÍ MINH HỒ ...,RLHKF5308SZ300075,0,other,hochiminh
2,1-3SFXBRK,TOÀN ANH,0908678461,TÂN THÔNG HỘI ẤP TÂN TIẾN CỦ CHI HỒ CHÍ MINH H...,RLHJF4601DY597619,0,other,hochiminh
3,1-1ZC6N3B,VĂN SỰ HỒ,0932622942,TỔ 9 ẤP 2A TÂN THẠNH ĐÔNG CỦ CHI HỒ CHÍ MINH H...,RLHKF4211MY015800,0,other,hochiminh
4,1-55F-14132,TRẦN ĐÌNH QUÂN,0933544794,47 22 KP4A TÂN THỚI HIỆP HỒ CHÍ MINH,RLHJF4611EY828738,0,other,hochiminh
...,...,...,...,...,...,...,...,...
211309,1-6A9CA3E,LÊ VĂN THẮNG,0988422595,ĐẠI LỘC TT BÚT SƠN HOẰNG HÓA THANH HÓA THANH HÓA,RLHJK0380RY047043,0,thanhhoa,thanhhoa
211310,1-GU5-3518,CHU THỊ HIỆP,0973055785,THÁNG 10 HOẰNG PHỤ THANH HÓA,RLHJK0347RZ412292,0,other,thanhhoa
211311,1-GVM-4759,HOÀNG THỊ QUYÊN,0388061120,Đ6 HOẰNG VINH THANH HÓA,RLHJF5804EY506671,0,other,thanhhoa
211312,1-FVS-8405,NGUYỄN THỊ NGA,0868389081,ÐÔNG XUÂN TRƯỜNG TRUNG THANH HÓA,RLHJF5800EY243577,0,other,thanhhoa


In [16]:
clean_address_output.to_csv(setts.tmp2_folder + '/cleaned_hms_city.csv',index=False)

In [17]:
# cleansing.to_csv("D:/UNIFICATION_PYTHON/cleansing.csv")